In [6]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.exceptions import NotFittedError
from sklearn.linear_model import LogisticRegression, LinearRegression


# ------------------ PAGE CONFIG ------------------
st.set_page_config(page_title="Causal Inference Analysis", layout="wide")


# ------------------ LOAD DATA ------------------
@st.cache_data
def load_data():
    try:
        data = pd.read_csv("sml_project/datasets/balanced_ihdata.csv")
        data = data.loc[:, ~data.columns.str.contains("^Unnamed")]  # remove unnamed cols
        data["treatment"] = data["treatment"].astype(float)
        return data
    except FileNotFoundError:
        st.warning("balanced_ihdata.csv not found. Using synthetic data instead.")
        return create_synthetic_data()


def create_synthetic_data(n=500, random_seed=42):
    np.random.seed(random_seed)
    data = pd.DataFrame({
        "treatment": np.concatenate([np.ones(n//2), np.zeros(n//2)]),
        "income": np.random.normal(50000, 10000, n),
        "birth_weight": np.random.normal(3000, 500, n),
        "parent_edu": np.random.randint(8, 18, n),
        "health_index": np.random.normal(70, 15, n),
        "housing_quality": np.random.normal(6, 2, n),
        "neighborhood_safety": np.random.normal(7, 2, n)
    })
    beta = np.array([0.5, 0.3, 0.8, 0.4, 0.6, 0.2])
    confounders = ["income", "birth_weight", "parent_edu", "health_index", "housing_quality", "neighborhood_safety"]
    X = data[confounders].values
    data["mu0"] = X.dot(beta) + np.random.normal(0, 1, n)
    data["mu1"] = data["mu0"] + 5 + 0.1 * data["income"]/10000 + 0.2 * data["parent_edu"]
    data["outcome_factual"] = np.where(data["treatment"] == 1, data["mu1"], data["mu0"])
    data["outcome_counterfactual"] = np.where(data["treatment"] == 1, data["mu0"], data["mu1"])
    for i in range(11, 26):
        data[f"x{i}"] = np.random.normal(0, 1, n)
    return data


if 'data' not in st.session_state or st.sidebar.button("Reset Data"):
    st.session_state.data = load_data()
    st.session_state.models_fitted = False

data = st.session_state.data
confounders = ["income", "birth_weight", "parent_edu", "health_index", "housing_quality", "neighborhood_safety"]

if 'scaler' not in st.session_state:
    st.session_state.scaler = StandardScaler().fit(data[confounders])


# ------------------ MODEL TRAINING ------------------
@st.cache_data(show_spinner=True)
def train_model(data, treatment_group, confounders):
    subset = data[data["treatment"] == treatment_group]
    if len(subset) == 0:
        raise ValueError(f"No data available for treatment group {treatment_group}")

    X, y = subset[confounders], subset["outcome_factual"]
    X_scaled = st.session_state.scaler.transform(X)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)

    y_pred = model.predict(X_scaled)
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)

    return model, r2, mse


def predict_outcome(model, input_data):
    try:
        input_data_scaled = st.session_state.scaler.transform(input_data)
        return model.predict(input_data_scaled)[0]
    except NotFittedError:
        st.error("Model not fitted yet.")
        return None


# ------------------ ATE CALCULATION ------------------
def calculate_ate(data):
    return data["mu1"].mean() - data["mu0"].mean()


def propensity_score_matching(data, confounders):
    X = data[confounders]
    y = data['treatment']
    propensity_model = LogisticRegression(max_iter=1000)
    propensity_model.fit(X, y)
    propensity_scores = propensity_model.predict_proba(X)[:, 1]

    treated = data[data['treatment'] == 1]
    control = data[data['treatment'] == 0]

    matched_outcomes, matched_ites = [], []

    for _, treated_row in treated.iterrows():
        x_treated = pd.DataFrame([treated_row[confounders].values], columns=confounders)
        treated_ps = propensity_model.predict_proba(x_treated)[:, 1]

        control_ps = propensity_model.predict_proba(control[confounders])[:, 1]
        distances = np.abs(control_ps - treated_ps)
        closest_control_idx = distances.argmin()

        treated_outcome = treated_row['outcome_factual']
        control_outcome = control.iloc[closest_control_idx]['outcome_factual']
        ite = treated_outcome - control_outcome

        matched_outcomes.append((treated_outcome, control_outcome))
        matched_ites.append(ite)

    return {
        'matched_ate': np.mean(matched_ites),
        'matched_outcomes': matched_outcomes,
        'pscore_balance': propensity_model.coef_[0]
    }


def doubly_robust_estimation(data, confounders):
    propensity_model = LogisticRegression(max_iter=1000)
    propensity_model.fit(data[confounders], data['treatment'])
    propensity_scores = propensity_model.predict_proba(data[confounders])[:, 1]

    outcome_model_treated = LinearRegression()
    outcome_model_control = LinearRegression()

    outcome_model_treated.fit(
        data[data['treatment'] == 1][confounders], 
        data[data['treatment'] == 1]['outcome_factual']
    )
    outcome_model_control.fit(
        data[data['treatment'] == 0][confounders], 
        data[data['treatment'] == 0]['outcome_factual']
    )

    dr_estimates = []

    for i in range(len(data)):
        # always keep DataFrame with correct column names
        row = data.loc[i, confounders]
        x = pd.DataFrame([row.values], columns=confounders)

        true_treatment = data.loc[i, 'treatment']
        outcome_treated = outcome_model_treated.predict(x)[0]
        outcome_control = outcome_model_control.predict(x)[0]
        ps = propensity_scores[i]

        dr_estimate = (
            true_treatment * (data.loc[i, 'outcome_factual'] - outcome_treated) / ps +
            (1 - true_treatment) * (data.loc[i, 'outcome_factual'] - outcome_control) / (1 - ps) +
            outcome_treated - outcome_control
        )
        dr_estimates.append(dr_estimate)

    return {
        'doubly_robust_ate': np.mean(dr_estimates),
        'dr_estimates': dr_estimates
    }


# ------------------ STREAMLIT APP ------------------
page = st.sidebar.radio("Navigate", ["Input & Results", "Dataset Overview", "Model Diagnostics"])

n_treated = (data["treatment"] == 1).sum()
n_control = (data["treatment"] == 0).sum()
st.sidebar.write(f"Dataset: {len(data)} samples")
st.sidebar.write(f"- Treated group: {n_treated}")
st.sidebar.write(f"- Control group: {n_control}")


try:
    if 'models_fitted' not in st.session_state or not st.session_state.models_fitted:
        with st.spinner("Training models..."):
            st.session_state.treated_model, st.session_state.treated_r2, st.session_state.treated_mse = train_model(data, 1, confounders)
            st.session_state.control_model, st.session_state.control_r2, st.session_state.control_mse = train_model(data, 0, confounders)
            st.session_state.models_fitted = True
except Exception as e:
    st.error(f"Error training models: {str(e)}")
    st.session_state.models_fitted = False


if page == "Input & Results":
    st.title("Causal Inference Analysis")

    col1, col2 = st.columns(2)
    with col1:
        income = st.slider("Income", float(data["income"].min()), float(data["income"].max()), float(data["income"].median()))
        birth_weight = st.slider("Birth Weight", float(data["birth_weight"].min()), float(data["birth_weight"].max()), float(data["birth_weight"].median()))
        parent_edu = st.slider("Parent Education", int(data["parent_edu"].min()), int(data["parent_edu"].max()), int(data["parent_edu"].median()))
    with col2:
        health_index = st.slider("Health Index", float(data["health_index"].min()), float(data["health_index"].max()), float(data["health_index"].median()))
        housing_quality = st.slider("Housing Quality", float(data["housing_quality"].min()), float(data["housing_quality"].max()), float(data["housing_quality"].median()))
        neighborhood_safety = st.slider("Neighborhood Safety", float(data["neighborhood_safety"].min()), float(data["neighborhood_safety"].max()), float(data["neighborhood_safety"].median()))

    treatment = st.radio("Treatment Applied?", ["Yes", "No"])
    treatment_value = 1 if treatment == "Yes" else 0

    if st.session_state.models_fitted:
        input_data = pd.DataFrame({
            "income": [income],
            "birth_weight": [birth_weight],
            "parent_edu": [parent_edu],
            "health_index": [health_index],
            "housing_quality": [housing_quality],
            "neighborhood_safety": [neighborhood_safety]
        })

        treated_outcome = predict_outcome(st.session_state.treated_model, input_data)
        control_outcome = predict_outcome(st.session_state.control_model, input_data)

        if treated_outcome is not None and control_outcome is not None:
            individual_treatment_effect = treated_outcome - control_outcome
            factual_outcome = treated_outcome if treatment_value else control_outcome
            counterfactual_outcome = control_outcome if treatment_value else treated_outcome

            psm_results = propensity_score_matching(data, confounders)
            dre_results = doubly_robust_estimation(data, confounders)

            col1, col2 = st.columns(2)
            with col1:
                st.metric("Population ATE", f"{calculate_ate(data):.4f}")
                st.metric("Your Factual Outcome", f"{factual_outcome:.4f}")
                st.metric("PSM ATE", f"{psm_results['matched_ate']:.4f}")
            with col2:
                st.metric("Counterfactual Outcome", f"{counterfactual_outcome:.4f}")
                st.metric("ITE", f"{individual_treatment_effect:.4f}")
                st.metric("Doubly Robust ATE", f"{dre_results['doubly_robust_ate']:.4f}")



2025-09-25 20:00:13.777 No runtime found, using MemoryCacheStorageManager
2025-09-25 20:00:13.785 No runtime found, using MemoryCacheStorageManager
